# Train and Evaluate Models

This notebook is used for training and evaluating different models for waste classification. It includes code for loading data, training each model separately, saving the trained models, and visualizing the training process.

In [1]:
import os
import torch
import matplotlib.pyplot as plt
from src.train import train_and_eval
from src.config import *  # Assuming config.py contains necessary configurations
from src.dataset import WasteDataset
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms
from tqdm import tqdm

# Load Data
train_dataset = WasteDataset(train, transform=train_transform)
val_dataset = WasteDataset(val, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Names
model_names = ['resnet18', 'resnet50', 'mobilenetv2', 'densenet121', 'efficientnet_b0']
results = []

num_classes = 24

for name in model_names:
    print(f"\n===== Training model: {name} =====")
    model = get_model(name, num_classes)
    acc, f1 = train_and_eval(model, train_loader, val_loader, device)
    results.append({'model': name, 'accuracy': acc, 'f1_score': f1})

# Save the trained model
for name in model_names:
    model_path = os.path.join('models', f'{name}_model.pth')
    torch.save(model.state_dict(), model_path)
    print(f"Saved model {name} to {model_path}")

# Plotting the results
accuracies = [result['accuracy'] for result in results]
f1_scores = [result['f1_score'] for result in results]

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar(model_names, accuracies, color='blue')
plt.title('Model Accuracies')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(model_names, f1_scores, color='orange')
plt.title('Model F1 Scores')
plt.xlabel('Model')
plt.ylabel('F1 Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()